# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 7 2022 2:57PM,239025,10,MET210974,"Methapharm, Inc.",Released
1,Jun 7 2022 2:57PM,239025,10,MET211012,"Methapharm, Inc.",Released
2,Jun 7 2022 2:57PM,239025,10,MET211022,"Methapharm, Inc.",Released
3,Jun 7 2022 2:57PM,239025,10,MET211023,"Methapharm, Inc.",Released
4,Jun 7 2022 2:57PM,239025,10,MET211024,"Methapharm, Inc.",Released
5,Jun 7 2022 2:55PM,239023,10,MET211010,"Methapharm, Inc.",Released
6,Jun 7 2022 2:55PM,239023,10,MET210993,"Methapharm, Inc.",Released
7,Jun 7 2022 2:55PM,239023,10,MET210994,"Methapharm, Inc.",Released
8,Jun 7 2022 2:55PM,239023,10,MET210995,"Methapharm, Inc.",Released
9,Jun 7 2022 2:55PM,239023,10,MET210996,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,239020,Released,2
35,239021,Released,1
36,239023,Released,21
37,239024,Released,12
38,239025,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239020,NaN,NaN,2.0
239021,NaN,NaN,1.0
239023,NaN,NaN,21.0
239024,NaN,NaN,12.0
239025,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238916,0.0,0.0,1.0
238940,0.0,17.0,13.0
238941,0.0,9.0,17.0
238942,0.0,5.0,12.0
238943,8.0,1.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238916,0,0,1
238940,0,17,13
238941,0,9,17
238942,0,5,12
238943,8,1,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238916,0,0,1
1,238940,0,17,13
2,238941,0,9,17
3,238942,0,5,12
4,238943,8,1,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238916,,,1
1,238940,,17,13
2,238941,,9,17
3,238942,,5,12
4,238943,8,1,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 7 2022 2:57PM,239025,10,"Methapharm, Inc."
5,Jun 7 2022 2:55PM,239023,10,"Methapharm, Inc."
26,Jun 7 2022 2:55PM,239024,10,"Methapharm, Inc."
38,Jun 7 2022 2:49PM,239021,10,Ivaoes Animal Health
39,Jun 7 2022 2:49PM,239020,10,"Nextsource Biotechnology, LLC"
41,Jun 7 2022 2:36PM,239019,10,ISDIN Corporation
75,Jun 7 2022 2:17PM,238998,20,"ACI Healthcare USA, Inc."
91,Jun 7 2022 2:16PM,239017,20,"ACI Healthcare USA, Inc."
98,Jun 7 2022 2:00PM,239016,18,Innova Softgel LLC
99,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 7 2022 2:57PM,239025,10,"Methapharm, Inc.",,,5
1,Jun 7 2022 2:55PM,239023,10,"Methapharm, Inc.",,,21
2,Jun 7 2022 2:55PM,239024,10,"Methapharm, Inc.",,,12
3,Jun 7 2022 2:49PM,239021,10,Ivaoes Animal Health,,,1
4,Jun 7 2022 2:49PM,239020,10,"Nextsource Biotechnology, LLC",,,2
5,Jun 7 2022 2:36PM,239019,10,ISDIN Corporation,,,34
6,Jun 7 2022 2:17PM,238998,20,"ACI Healthcare USA, Inc.",,,16
7,Jun 7 2022 2:16PM,239017,20,"ACI Healthcare USA, Inc.",,,7
8,Jun 7 2022 2:00PM,239016,18,Innova Softgel LLC,,,1
9,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc.",,,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 2:57PM,239025,10,"Methapharm, Inc.",5,,
1,Jun 7 2022 2:55PM,239023,10,"Methapharm, Inc.",21,,
2,Jun 7 2022 2:55PM,239024,10,"Methapharm, Inc.",12,,
3,Jun 7 2022 2:49PM,239021,10,Ivaoes Animal Health,1,,
4,Jun 7 2022 2:49PM,239020,10,"Nextsource Biotechnology, LLC",2,,
5,Jun 7 2022 2:36PM,239019,10,ISDIN Corporation,34,,
6,Jun 7 2022 2:17PM,238998,20,"ACI Healthcare USA, Inc.",16,,
7,Jun 7 2022 2:16PM,239017,20,"ACI Healthcare USA, Inc.",7,,
8,Jun 7 2022 2:00PM,239016,18,Innova Softgel LLC,1,,
9,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc.",8,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 2:57PM,239025,10,"Methapharm, Inc.",5,,
1,Jun 7 2022 2:55PM,239023,10,"Methapharm, Inc.",21,,
2,Jun 7 2022 2:55PM,239024,10,"Methapharm, Inc.",12,,
3,Jun 7 2022 2:49PM,239021,10,Ivaoes Animal Health,1,,
4,Jun 7 2022 2:49PM,239020,10,"Nextsource Biotechnology, LLC",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 2:57PM,239025,10,"Methapharm, Inc.",5.0,NaN,NaN
1,Jun 7 2022 2:55PM,239023,10,"Methapharm, Inc.",21.0,NaN,NaN
2,Jun 7 2022 2:55PM,239024,10,"Methapharm, Inc.",12.0,NaN,NaN
3,Jun 7 2022 2:49PM,239021,10,Ivaoes Animal Health,1.0,NaN,NaN
4,Jun 7 2022 2:49PM,239020,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 2:57PM,239025,10,"Methapharm, Inc.",5.0,0.0,0.0
1,Jun 7 2022 2:55PM,239023,10,"Methapharm, Inc.",21.0,0.0,0.0
2,Jun 7 2022 2:55PM,239024,10,"Methapharm, Inc.",12.0,0.0,0.0
3,Jun 7 2022 2:49PM,239021,10,Ivaoes Animal Health,1.0,0.0,0.0
4,Jun 7 2022 2:49PM,239020,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2629065,87.0,0.0,0.0
12,238984,1.0,1.0,0.0
15,716824,45.0,27.0,8.0
18,478026,2.0,0.0,0.0
19,716929,1.0,2.0,0.0
20,1911881,139.0,22.0,0.0
21,716921,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2629065,87.0,0.0,0.0
1,12,238984,1.0,1.0,0.0
2,15,716824,45.0,27.0,8.0
3,18,478026,2.0,0.0,0.0
4,19,716929,1.0,2.0,0.0
5,20,1911881,139.0,22.0,0.0
6,21,716921,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,87.0,0.0,0.0
1,12,1.0,1.0,0.0
2,15,45.0,27.0,8.0
3,18,2.0,0.0,0.0
4,19,1.0,2.0,0.0
5,20,139.0,22.0,0.0
6,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,87.0
1,12,Released,1.0
2,15,Released,45.0
3,18,Released,2.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21
Status,,,,,,,
Completed,0.0,0.0,8.0,0.0,0.0,0.0,0.0
Executing,0.0,1.0,27.0,0.0,2.0,22.0,0.0
Released,87.0,1.0,45.0,2.0,1.0,139.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21
0,Completed,0.0,0.0,8.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,27.0,0.0,2.0,22.0,0.0
2,Released,87.0,1.0,45.0,2.0,1.0,139.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21
0,Completed,0.0,0.0,8.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,27.0,0.0,2.0,22.0,0.0
2,Released,87.0,1.0,45.0,2.0,1.0,139.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()